In [2]:
# ✅ Set Gemini API key securely (Google Colab specific)
import os

from dotenv import load_dotenv

load_dotenv()
gemini_api_key = os.getenv("GOOGLE_API_KEY")

from langchain_google_genai import ChatGoogleGenerativeAI

from app.prompts import COURSE_PLANNER
from app.schema import LearningPlan

In [6]:
from trustcall import create_extractor

llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash", temperature=0.5, max_retries=2, api_key=gemini_api_key)
# llm = llm.with_structured_output(LearningPlan)


llm = create_extractor(
    llm,
    tools=[LearningPlan],
    tool_choice="any",
    enable_inserts=True,
)


response = llm.invoke([("system", COURSE_PLANNER), ("user", "generative AI & its advances (14 yr old)")])

print(response)

Key '$defs' is not supported in schema, ignoring
Key 'title' is not supported in schema, ignoring


{'messages': [AIMessage(content='', additional_kwargs={'function_call': {'name': 'LearningPlan', 'arguments': '{"topic": "Generative AI for a 14-year-old", "weekly_plans": [{"activities": [{"frequency": "Daily", "description": "Read introductory articles on Generative AI."}, {"frequency": "3 times a week", "description": "Watch videos explaining what Generative AI is and how it works."}, {"frequency": "2 times a week", "description": "Explore different types of Generative AI models (GANs, VAEs, Transformers)."}], "focus": "Understanding the Basics of Generative AI", "week_number": 1.0, "checkpoint": "Write a short paragraph explaining what Generative AI is in your own words.", "resources": [{"type": "documentation", "name": "What is Generative AI?", "url": "https://www.ibm.com/topics/generative-ai"}, {"url": "https://www.youtube.com/watch?v=69FGwAI1Jcs", "name": "Generative AI, Explained", "type": "other"}]}, {"activities": [{"frequency": "Daily", "description": "Experiment with online

In [ ]:
# ✅ 🔧 MISSING FUNCTION: generate_learning_path()
def generate_learning_path(topic):
    prompt = prompt_template.format(topic=topic)
    try:
        response = llm.invoke(prompt)
        # Extract content properly from Gemini response
        return response.content if hasattr(response, "content") else str(response)
    except Exception as e:
        return f"Error generating learning path: {e}"


# ✅ Optional test to see if Gemini works (can comment out later)
print(generate_learning_path("Python"))

In [ ]:
# ✅ Gradio interface
import gradio as gr


def chatbot(topic):
    if not topic.strip():
        return "Please enter a topic."
    return generate_learning_path(topic)


def download_learning_path(topic):
    plan = generate_learning_path(topic)
    filename = f"{topic}_learning_path.txt"
    with open(filename, "w") as f:
        f.write(plan)
    return filename


with gr.Blocks() as demo:
    gr.Markdown("## 📘 Gemini AI Learning Path Generator")

    with gr.Row():
        topic_input = gr.Textbox(label="Enter a Topic")
        generate_btn = gr.Button("Generate Learning Path")

    output = gr.Textbox(label="Learning Path", lines=20)
    download_btn = gr.Button("Download")

    generate_btn.click(chatbot, inputs=topic_input, outputs=output)
    download_btn.click(download_learning_path, inputs=topic_input, outputs=None)

demo.launch(share=True, debug=True)